RUNS with 83.33 Wacc ACCURACY with Mic input-Final project




In [ ]:
!pip install -q transformers datasets torchaudio librosa
!pip install -q gradio
!pip install -q gTTS


!sudo apt-get install -y sox libsndfile1

print("Dependencies installed successfully.")



import json
import os
from gtts import gTTS
import pandas as pd
import gradio as gr


DATASET_JSON = [
	{"input":"14 மாசம் கழிச்சு, ரெண்டாவது calf பிறந்துடுச்சு.","output":"Fourteen months later, a second calf was born."},
	{"input":"Chidambaram-க்காக ஆஜர் ஆன Senior advocate Kapil Sibal, ஒரு MP-க்கு இந்த condition சரியில்லைன்னு சொல்லிட்டு, அவர் எங்கயும் ஓட மாட்டார்-ன்னு சொன்னாரு.","output":"Senior advocate Kapil Sibal, who appeared for Chidambaram, said that this condition is not right for an MP and he will not run anywhere."},
	{"input":"இந்த photo அப்போ எடுத்தது.","output":"This photo was taken then."},
	{"input":"இதுவரைக்கும் JWG meeting-ல ரெண்டு round நடந்துருக்கு.","output":"So far, there have been two rounds at the JWG meeting."},
	{"input":"இந்த உலக வாழ்க்கை என்பது விளையாட்டு மட்டும் தான், மறுமை வாழ்க்கை தான் பயபக்தியுள்ளவர்களுக்கு நல்லது. உங்க reason-ஐ use பண்ணிக்கலையா?","output":"The life of this world is only a game, and the life to come is good for the Godwary. Do you use your reason?"},
	{"input":"அதிகபட்ச வெப்பநிலை 35 degrees Celsius-ஆ இருக்கும், குறைந்தபட்ச வெப்பநிலை 27 degree-ஆ இருக்கும்னு சொல்லி இருக்காங்க.","output":"The maximum temperature will be around 35 degrees Celsius and the minimum temperature will be around 27 degrees Celsius."},
	{"input":"இவ்வளவு வாழ்க்கைகள்லயும், நீங்க எதைத் தேர்ந்தெடுக்குறீங்கன்னு சொல்ல முடியாது. ஆனா, உங்க சொந்த experience நிறைய விஷயங்களுக்கு ஒரு colour-அ கொடுக்கும். அது எப்படி இருக்கும்- ஒருத்தர top-லயும், ஒருத்தர bottom-லயும். அப்படி ஒரு sibling அல்லது child-ஆ, உங்க experience ரொம்ப painful-ஆ இருந்தா, அவங்கள bottom-லயே push பண்ணிடுவீங்க.","output":"In all these lives, you can't tell what you choose. But your own experience can give a lot of things a colour. What does it look like - one on the top, one at the bottom. If you're a sibling or a child, if your experience is too painful, you're pushing them to the bottom."},
	{"input":"speaker சொன்ன ஒரு விஷயத்த அவங்க agree பண்ணா, hum பண்ணுவாங்க இல்லனா grunted பண்ணுவாங்க.","output":"If they agree with what the speaker says, they will hum or grunted."},
	{"input":"இந்த region-ல விவசாயிகள் அதிகமா paddy-ம் wheat-ம் பயிர் பண்றாங்க.","output":"Farmers in this region grow paddy and wheat."},
	{"input":"இதோட, ஒரு video-ம் release பண்ணிருக்காங்க.","output":"Along with this, a video has also been released."},
	{"input":"காச நோய்க்கு அறிகுறிகள்","output":"Symptoms of tuberculosis"},
	{"input":"இந்த meeting-ல கீழே உள்ள decisions எடுக்கப்பட்டது:","output":"In this meeting, the decisions taken below:"},
	{"input":"அஷ்டமி நாட்களில் பைரவாருக்கு சிறப்பு அபிஷேகங்கள் செய்யப்படும்.","output":"Special abishekam is performed to Lord Bhairava on Ashtami days."},
	{"input":"உங்க தவறு இல்ல.","output":"It's not your fault."},
	{"input":"இதனால Jayanagar-ல election postpone பண்ணிட்டாங்க.","output":"So they held an election postpone in Jayanagar."},
	{"input":"அந்த சந்தோஷம் எதுல இருந்து வந்தது?","output":"Where did that happiness come from?"},
	{"input":"Maharashtra BJP MP Nana Patole party-யையும் Lok Sabha-யும் விட்டு விலகிட்டாரு.","output":"He has left the Maharashtra BJP MP Nana Patole Party and the Lok Sabha."},
	{"input":"அவங்க picture-க்கு Sky above, sand below, peace within -ன்னு caption போட்டாங்க.","output":"She captioned the picture as Sky above, sand below, peace within."},
	{"input":"பகவான் சிவபெரியா வழிபாடு பண்ணுங்க.","output":"Worship Lord Sivaperiya."},
	{"input":"'Virunthinar Potruthum, Virunthinar Potruthum' (care for the guest) scheme-ல tourism related industry-ல வேலை செய்றவங்களுக்கு hospitality culture-அ inculcating பண்றது, adventure tourism, rural tourism, eco-tourism, இதெல்லாம் develop பண்றது மாதிரி நிறைய புது initiatives எடுக்கப்பட்டிருக்கு.","output":"In the 'Virunthinar Potruthum, Virunthinar Potruthum' (care for the guest) scheme, many new initiatives have been taken to develop hospitality culture or inculcating, adventure tourism, rural tourism, eco-tourism, etc."},
	{"input":"Lawrence Bowen, Ephesus-ல Paul பண்ண public ministry பத்தின Acts 20: 20-ல apostle Paul சொன்னதுக்கு point பண்ணி சொன்னாரு, Paul சாட்சியம் சொல்ல எந்த opportunity-யும் use பண்ணாருன்னு சொல்லி.","output":"Lawrence Bowen, in Acts 20:20 about the public ministry by Paul in Ephesus, pointed out what Paul said about the public ministry, saying that Paul used any opportunity to testify."},
	{"input":"அவங்களோட பலவீனமானவங்க மேல arrogance-ஆ இருந்த சில முக்கிய ஆளுங்க கேட்டாங்க: Saleh-அ அவரோட Lord-ஆல் அனுப்பப்பட்டிருக்காருன்னு உங்களுக்குத் தெரியுமா? நிச்சயமா அவர் சொல்லியிருக்க message-அ நம்புறோம்னு சொன்னாங்க.","output":"Some prominent people who were arrogance of their weak asked: Did you know that Saleh has been sent by his Lord? Surely, he said he would believe the message."},
	{"input":"இன்னொரு தீர்க்கதரிசனோட வார்த்தைகள்ல கோபப்பட்ட King Jehoiakim, அந்தத் தீர்க்கதரிசியான Urijah-ஐ கொன்று போட்டாரு. Jeremiah 26: 21 - 24.","output":"Angered by the words of another prophet, King Jehoiakim killed the prophet Urijah. Jeremiah 26: 21 - 24."},
	{"input":"President i Jinping, Chennai-ல நடந்த 2nd Informal Summit-ல PM-ஆட host பண்ணதுக்கு ரொம்ப நன்றி சொன்னாரு, PM Modi-யும் இந்திய மக்களும் கொடுத்த வரவேற்ப மறக்க முடியாதுன்னு சொன்னாரு.","output":"President i Jinping thanked me for hosting the PM-I at the 2nd Informal Summit in Chennai and said that he will never forget the welcome given by PM Modi and the people of India."},
	{"input":"இத பாத்து fans எல்லாம் ரொம்ப சந்தோஷப்பட்டாங்க.","output":"Fans were very happy to see this."},
	{"input":"Bible-ல சொல்லி இருக்காங்க, சீக்கிரமே கடவுள் எல்லா அநீதத்தையும் நீக்கிடுவாருன்னு. 2 Peter 3: 7 - 9, 13 படிக்கவும்.","output":"As the Bible says, God will soon remove all injustice. Read 2 Peter 3:7-9, 13."},
	{"input":"குடிநீர் பிரச்சனைய சமாளிக்க அரசாங்கம் புது Jal Shakti அமைச்சகம் உருவாக்கி இருக்கு.","output":"The government has created a new Jal Shakti Ministry to tackle the drinking problem."},
	{"input":"தேங்காய் எண்ணெய் சூடாக்கி, கடுகு சேர்த்து, வெங்காயம், மிளகுத்தூள், இஞ்சி, பூண்டு சேர்த்து வதக்கி விடுங்க.","output":"Heat coconut oil, add mustard seeds, onions, peppercorns, ginger and garlic."},
	{"input":"அது உண்மையிலேயே சரியானது இல்ல.","output":"That's not really right."},
	{"input":"என் economic situation-ஐப் பாத்தா, கொஞ்சம் secular work பண்ணா நிறைய சம்பாதிக்க முடியும்னு சில சமயம் தோணும்.","output":"Looking at my economic situation, sometimes I feel that I can earn a lot if I do a little secular work."},
	{"input":"ஸ்கூல்-ல என்ன படிச்சீங்க?","output":"What did you study in school?"},
	{"input":"அந்த event-ல, ரஜினிகாந்த் அவர்களுக்கு Icon of Golden Jubilee award கொடுத்து honour பண்ணாங்க.","output":"At that event, Rajinikanth was honoured with the Icon of Golden Jubilee award."},
	{"input":"நீங்க splosh பண்ணுவீங்களா?","output":"Do you splosh?"},
	{"input":"மேல இருக்கு.","output":"It's up."},
	{"input":"இந்திய பிரதமர் நரேந்திர மோடி அவர்கள், காஷ்மீர் பிரச்சினையில் எனக்கு உதவி பண்ண சொன்னாங்க. எனக்கு mediator ஆகணும்னு ரொம்ப ஆசை, -ன்னு Trump சொல்லி இருக்காரு.","output":"Indian Prime Minister Narendra Modi asked me to help him on the Kashmir issue. \"I want to be a mediator,\" Trump said."},
	{"input":"எனக்கு Telugu language ரொம்பப் பிடிக்கும்.","output":"I love the Telugu language."},
	{"input":"கன்னியாகுமரி மாவட்டம் மற்றும் தென்காசி மாவட்டத்தின் Shencottah தாலுகாவிலுள்ள Perumkollar","output":"Perumkollar in Kanniyakumari District and Shencottah Taluk of Tenkasi District"},
	{"input":"( கோடி ரூபாய்ல )","output":"(Crores of rupees)"},
	{"input":"ஜம்மு காஷ்மீரில் உள்ள ஸ்ரீ அமர்நாத் ஜி ஹோலி குகைல பிரார்த்தனை செய்த பிறகு மிகவும் ஆசீர்வாக இருக்கு, என்று அவர் ட்விட்டரில் எழுதினார்.","output":"So blessed after offering prayers at Shri Amarnath Ji Holi cave in J K,\" he wrote on Twitter."},
	{"input":"William Brown Street-அ சுத்தி இருக்குற இடத்த 'Cultural Quarter' -ன்னு சொல்லுவாங்க, ஏன்னா அங்க நிறைய civic buildings இருக்கு, William Brown Library, Walker Art Gallery, Picton Reading Rooms, World Museum Liverpool இதெல்லாம் அதுல அடங்கும்.","output":"The area around William Brown Street is called the 'Cultural Quarter' because it has many civic buildings, including the William Brown Library, the Walker Art Gallery, the Picton Reading Rooms, and the World Museum Liverpool."},
	{"input":"GST implementation shortfall-ஐ சரிக்கட்ட Kerala மற்றும் West Bengal -ன்னு இன்னும் ரெண்டு States Option-1-ஐ தேர்ந்தெடுத்துள்ளாங்க.","output":"Two more states, Kerala and West Bengal, have opted for Option 1 to compensate for the GST implementation shortfall."},
	{"input":"ஆனா தமிழ் சினிமாவுல இது புதுசு.","output":"But this is new in Tamil cinema."},
	{"input":"சிறிது நேரம் கழிச்சு, அவர் re","output":"After a while, he re"},
	{"input":"இது முழுக்க முழுக்க continents-அயே அழிச்சுடலாம் அப்படிங்கற பயம், இப்படி ஒரு comeback possible-ஆ இருக்குமானு கேக்குதுக்கு நம்மள தூண்டும்.","output":"The fear of extinction of continents makes us wonder if there is a comeback possible."},
	{"input":"திரும்பி வந்துட்டேன்.","output":"I'm back."},
	{"input":"அவங்க players நல்லா விளையாண்டாங்க.","output":"Their players played well."},
	{"input":"இந்த சம்பவம் Nalgonda town-ல நடந்துச்சு.","output":"The incident took place in Nalgonda town."},
	{"input":"அந்த message WhatsApp-லயும் Facebook-லயும் பகிரப்பட்டுருக்கு.","output":"The message has been shared on WhatsApp and Facebook."},
	{"input":"அவங்க fight-ல இருந்து தப்பிச்சது மட்டும் இல்லாம, தன்னோட பாதுகாப்புக்காகவும் steps எடுத்திருக்காங்க.","output":"Not only did she survive the fight, but she took steps to protect herself."},
	{"input":"அது தெய்வீகமானது.","output":"It is divine."},
	{"input":"25 லட்சம் ரிச்சாபடிக்கப்பட்டுருக்கு.","output":"Rs 25 lakh has been stolen."},
	{"input":"வேற யாராச்சும் இருக்காங்களா?","output":"Is there anyone else?"},
	{"input":"அதான் நானே இந்த படத்த produce பண்ணலாம்னு முடிவு பண்ணேன்.","output":"That's why I decided to produce this film myself."},
	{"input":"இப்போ அவங்களால state and central governments-ஓட public welfare schemes-அ use பண்ணிக்க முடியும். இந்த agreement-ஐ Prime Minister special -ன்னு சொன்னாரு, ஏன்னா இது cooperative federalism-உடைய spirit-ஐ symbolise பண்ணுதுன்னு சொன்னாரு.","output":"Now they can use public welfare schemes of state and central governments. The Prime Minister called this agreement special because it symbolises the spirit of cooperative federalism."},
	{"input":"நாம கொடுக்க மாட்டேன்னு சொல்ல முடியாதுங்க.","output":"We can't say we won't give."},
	{"input":"அவங்கல நிறைய பேர எனக்கு தெரியும்.","output":"I know a lot of them."},
	{"input":"அவங்க மனைவி கர்ப்பா இருக்காங்க.","output":"His wife is pregnant."},
	{"input":"நாங்க இந்தியாவுல இருக்கோம்.","output":"We are in India."},
	{"input":"TNEA rank list 2020: எல்லா students-க்கும் merit basis-ல rank கொடுக்கப்படுவாங்க.","output":"TNEA rank list 2020: All students will be given rank on merit basis."},
	{"input":"ஆனா, எந்த prizes-ம் கிடையாதுங்க.","output":"But there are no prizes."},
	{"input":"என் வேலை என்னன்னா, குழந்தைகளுக்கு சந்தோஷத்த கொண்டு வரணும்னு தான்.","output":"My job is to bring happiness to the children."},
	{"input":"அவர் இன்னும் இங்க வரல.","output":"He hasn't been here yet."},
	{"input":"அவர பிடிக்க போலீஸ் முயற்சி பண்ணிட்டு இருக்காங்க.","output":"The police are trying to catch him."},
	{"input":"Cocoa 100 கிராம்","output":"Cocoa 100 g"},
	{"input":"உலக கிரிக்கெட்ல ரொம்பவே சிறந்த review takers-ல MS Dhoni ஒருவர்.","output":"MS Dhoni is one of the best review takers in world cricket."},
	{"input":"சகோதரர்களே, எல்லாரும் என்னோட followers-ஆ இருக்கணும், நான் உங்களுக்கு ஒரு ensample-ஆ இருக்கேன்னு நினைச்சு, யாரு எப்படி நடந்துக்கனும்னு கவனிங்க.","output":"Brothers, everyone should be my followers, think I'm an ensample to you, and watch who should behave the way."},
	{"input":"ரொம்பவே rationing இருந்துச்சு.","output":"It was very rationing."},
	{"input":"இதனால, தண்ணி waste ஆகுது.","output":"As a result, water becomes wasted."},
	{"input":"Babylon the Great பத்தின Revelation-ல உள்ள தீர்க்கதரிச்சத்தோட in - depth study-க்கு, 1988-ல் Watchtower Bible and Tract Society of New York, Inc. publish பண்ணின Revelation Its Grand Climax At Hand! புத்தகத்தோட 33 - 37 chapters பாருங்க.","output":"For an in-depth study of Babylon the Great, Revelation was published by the Watchtower Bible and Tract Society of New York, Inc. in 1988. Its Grand Climax At Hand! Look at chapters 33-37 of the book."},
	{"input":"ஆனா, rules அப்படித்தான் இருக்கும்.","output":"But the rules are like that."},
	{"input":"பாதிக்கப்பட்டவருக்கு தலையிலும் மார்பு பகுதியிலும் காயம் ஏற்பட்டிருக்கு.","output":"The victim sustained injuries to his head and chest."},
	{"input":"அவர் மகிழ்ச்சியா agree பண்ணிட்டாங்க.","output":"He happily agreed."},
	{"input":"DMK-ல Karunanidhi-யோட அண்ணன் Alagiri தான் முதல்ல இருக்காரு.","output":"Karunanidhi's brother Alagiri is the first in the DMK."},
	{"input":"மனிதர்கள் இயல்பிலேயே அபிதானமான பாவம்கூடவர்கள் ஆகையால், இயேசுவோட வழியில நடக்குறது அவங்க normal way of walking கிடையாது.","output":"Since humans are absurdly sinful by nature, walking in Jesus' way is not their normal way of walking."},
	{"input":"Eugene Fama மாதிரி academics சொல்றது என்னன்னா, technical analysis-க்கு evidence ரொம்ப கம்மிதான், efficient-market hypothesis-ஓட weak form-க்கும் இதுக்கும் ஒத்து போற மாதிரி இல்லன்னு.","output":"Academics like Eugene Fama say that the evidence for technical analysis is lacking and does not match the weak form of efficient market hypothesis."},
	{"input":"Notification official website-ல இருக்கும்.","output":"The notification is available on the official website."},
	{"input":"போர் நடக்கும்.","output":"War will happen."},
	{"input":"ரெண்டாவதா, நம்ம வாழ்க்கையில நல்லா போயிட்டு இருக்குற areas-ல மட்டும் focus பண்ணிட்டு, work பண்ண வேண்டிய aspects-அ ignore பண்ணிடக் கூடாது.","output":"Second, we should not focus only on the areas that are going well in our lives and ignore the aspects that need to be worked."},
	{"input":"அப்படி இருந்தாலும், சில விஷயத்துல உங்க view-அ மாத்தி பாக்க வேண்டியதா இருக்கலாம்.","output":"Even so, you may need to change your view in some cases."},
	{"input":"அவங்க Maur Mandi-ல போலீஸ்கிட்ட சொல்லிட்டாங்க.","output":"They told the police at Maur Mandi."},
	{"input":"ஆனா, France-லயும் Netherlands-லயும் voters இத overwhelmingly reject பண்ணிட்டாங்க - அதனால EU-க்குள்ள ஒரு crisis வந்தது, அதனால Brussels summit-ல budget-அ agree பண்ண முடியாம போச்சு, constitution ratification process-அ indefinitely நிறுத்திட்டாங்க.","output":"But voters in France and the Netherlands overwhelmingly rejected it - so there was a crisis in the EU, which made it impossible to agree to the budget at the Brussels summit and stopped the constitution ratification process indecently."},
	{"input":"இதோ சில simple tips.","output":"Here are some simple tips."},
	{"input":"போலீஸ் accident-ஆ register பண்ணி, அதுக்கு investigate பண்ணிட்டு இருக்காங்க.","output":"The police have registered an accident and are investigating it."},
	{"input":"எத்தனை raiders?","output":"How many raiders?"},
	{"input":"ஆனா, wicked one- கிட்ட இருந்து நம்மளை மீட்பதுக்கு Jehovah- மேல நம்பிக்கை வச்சு, அவர் மேல loyalty-ஆ இருக்க முடிவு பண்ணிக்கணும்.","output":"But to rescue ourselves from the wicked one, we must trust in Jehovah and decide to be loyalty to him."},
	{"input":"ஆனா, தெய்வீக வழங்கப்பட்ட கொள்கை, விதி அல்லது சட்டம் இல்லாதபோது, முற்றிலும் தனிப்பட்ட விஷயங்களிலும் நம்மளோட மனசாட்சி-யோட தீர்ப்புகளை சக Christians-க மேல திணிக்கிறது தப்பானதுன்னு மனசுல எப்பவும் வச்சுக்கணும். ரோமாஸ் 14: 1 - 4. Galatians 6: 5.","output":"But we should always bear in mind that it is wrong to impose our conscience's judgments on fellow Christians, even in purely personal matters, when there is no divinely given principle, rule, or law. Romas 14: 1 - 4. Galatians 6: 5."},
	{"input":"நாடு முழுவதும் நடந்த exam-ல மொத்தம் 15,19,375 students register பண்ணிருந்தாங்க.","output":"A total of 15,19,375 students had registered for the exam across the country."},
	{"input":"State Pollution Board-ல இருந்து எந்த certificate-ம் வாங்கலேன்னு சொல்லி இருக்காங்க.","output":"They said they didn't get any certificate from the State Pollution Board."},
	{"input":"இனிமேல் என்ன சொல்லுறதுன்னே எனக்குத் தெரியல.","output":"I don't know what to say anymore."},
	{"input":"Treatment-ல என்ன risks இருக்கு?","output":"What are the risks of treatment?"},
	{"input":"BJP-ல இருக்கவங்க Shyama Prasad Mukherjee-ஓட தலையாய்ப்பணத்த மறந்துட்டாங்க.","output":"People in the BJP have forgotten Shyama Prasad Mukherjee's head money."},
	{"input":"இது ஒரு நாள் கண்டிப்பா நடக்கும்.","output":"This will definitely happen one day."},
	{"input":"இதுக்கு எதிரா பெண்கள் ரொம்ப கஷ்டப்பட்டு fight பண்ணிருக்காங்க.","output":"Women have fought hard against this."},
	{"input":"பூகம்பத்துல இறந்த ஒவ்வொருத்தருக்கும், அவங்க families-க்கு Rs. அளிக்கப்படுது.","output":"For every earthquake victim, their families are given Rs."},
	{"input":"ஆனா அவர் அப்படி பண்ணல.","output":"But he didn't."},
	{"input":"தமிழ்நாட்டுல தண்ணி பஞ்சம் ரொம்ப முக்கியமான பிரச்சனையா இருக்கு.","output":"Water scarcity is a major problem in Tamil Nadu."},
	{"input":"நம்ம Kingdom message-அ தொடர்ந்து சொல்லிட்டே இருக்க தைரியம் ரொம்ப முக்கியம். சில பேர், உறவினர் கூட, உங்களை கேலியாக்கலாம், இல்லன்னா வேற ஏதாவது விதத்துல discourage பண்ண try பண்ணுவாங்க. அப்போலர் பால் அவர்கள் தன்னுடைய ஊழியத்தை உண்மையுள்ளவர்களா நிறைவேற்றியதால், அவர் மேல ஒரு தடவைக்கு மேல உடல் ரீதியா தாக்கப்பட்டார்.","output":"Courage is very important to keep telling our Kingdom message. Some people, even relatives, may make fun of you or try to discourage you in some other way. Apollo Paul was physically attacked more than once because he had faithfully fulfilled his ministry."},
	{"input":"Maximum...","output":"Maximum..."},
	{"input":"ISLAMABAD Pakistan Prime Prime Minister Imran Khan, powerful spy agency-யோட headquarters-க்கு first time visit பண்ணும்போது, ISI-அ நம்ம first line of defence-ன்னு சொல்லி இருக்காரு.","output":"ISLAMABAD Pakistan Prime Minister Imran Khan, during his first visit to the headquarters of the Powerful Spy Agency, said that ISI is our first line of defence."},
	{"input":"இந்த படத்தோட bankroll பண்ணது Vijaya Productions, music Santhosh Narayanan.","output":"The film is bankrolled by Vijaya Productions, music Santhosh Narayanan."}
]























   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1 sox
0 upgraded, 7 newly installed, 0 to remove and 41 not upgraded.
Need to get 617 kB of archives.
After this operation, 1,764 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrnb0 amd64 0.1.5-1 [94.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrwb0 amd64 0.1.5-1 [49.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe 

In [ ]:
df = pd.DataFrame(DATASET_JSON)
df.rename(columns={'input': 'tanglish_text', 'output': 'english_transcription'}, inplace=True)


AUDIO_DIR = "simulated_audio"
os.makedirs(AUDIO_DIR, exist_ok=True)


sample_df = df.head(5)
sample_df['audio_path'] = ''

print("Generating simulated audio files (this may take a moment)...")


for index, row in sample_df.iterrows():
    try:
        tanglish = row['tanglish_text']

        tts = gTTS(tanglish, lang='ta')
        filename = os.path.join(AUDIO_DIR, f"sample_{index}.mp3")
        tts.save(filename)
        sample_df.loc[index, 'audio_path'] = filename
    except Exception as e:
        print(f"Could not generate audio for row {index}: {e}")

print("\nSimulated audio data (Tanglish to English pairs) is ready for the demo:")
print(sample_df[['audio_path', 'tanglish_text', 'english_transcription']])



/tmp/ipython-input-3009121145.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['audio_path'] = ''


Generating simulated audio files (this may take a moment)...

Simulated audio data (Tanglish to English pairs) is ready for the demo:
                     audio_path  \
0  simulated_audio/sample_0.mp3   
1  simulated_audio/sample_1.mp3   
2  simulated_audio/sample_2.mp3   
3  simulated_audio/sample_3.mp3   
4  simulated_audio/sample_4.mp3   

                                       tanglish_text  \
0    14 மாசம் கழிச்சு, ரெண்டாவது calf பிறந்துடுச்சு.   
1  Chidambaram-க்காக ஆஜர் ஆன Senior advocate Kapi...   
2                         இந்த photo அப்போ எடுத்தது.   
3  இதுவரைக்கும் JWG meeting-ல ரெண்டு round நடந்து...   
4  இந்த உலக வாழ்க்கை என்பது விளையாட்டு மட்டும் தா...   

                               english_transcription  
0     Fourteen months later, a second calf was born.  
1  Senior advocate Kapil Sibal, who appeared for ...  
2                         This photo was taken then.  
3  So far, there have been two rounds at the JWG ...  
4  The life of this world is only a game, a

break 2 7.11.25

In [ ]:
!pip install -q transformers datasets torchaudio librosa gradio gTTS jiwer

import os
import json
import torch
from transformers import pipeline
import gradio as gr
from jiwer import wer

# ---------------------------
# MODEL INITIALIZATION
# ---------------------------
MODEL_NAME = "openai/whisper-medium"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device
)

print(f"✅ ASR model loaded: {MODEL_NAME} (device: {device})")

# ---------------------------
# FEEDBACK STORAGE
# ---------------------------
FEEDBACK_FILE = "asr_feedback_log.json"

if os.path.exists(FEEDBACK_FILE):
    with open(FEEDBACK_FILE, "r") as f:
        feedback_history = json.load(f)
else:
    feedback_history = {"records": []}

def calculate_overall_accuracy():
    records = feedback_history.get("records", [])
    if not records:
        return 0.0
    total = len(records)
    correct = sum(1 for r in records if r["correct"])
    return (correct / total) * 100

def save_feedback_record(is_correct, wer_val, wacc_val):
    feedback_history["records"].append({
        "correct": is_correct,
        "wer": wer_val,
        "wacc": wacc_val
    })
    with open(FEEDBACK_FILE, "w") as f:
        json.dump(feedback_history, f)

def get_live_accuracy_summary():
    total_samples = len(feedback_history["records"])
    correct_samples = sum(1 for r in feedback_history["records"] if r["correct"])
    avg_accuracy = calculate_overall_accuracy()
    return total_samples, correct_samples, avg_accuracy

# ---------------------------
# MAIN FUNCTION
# ---------------------------
def transcribe_and_evaluate(audio_file_path, reference_text=None):
    if not audio_file_path:
        return "Please upload or record audio.", "", ""
    try:
        result = asr_pipeline(audio_file_path, task="translate", language="ta")
        transcription = result["text"].strip()

        if reference_text and reference_text.strip():
            error_rate = wer(reference_text.lower(), transcription.lower())
            word_accuracy = (1 - error_rate) * 100
            if word_accuracy < 80:
                save_feedback_record(False, error_rate, word_accuracy)
                return transcription, f"❌ WAcc: {word_accuracy:.2f}% (below 80%)", ""
            else:
                save_feedback_record(True, error_rate, word_accuracy)
                return transcription, f"✅ WAcc: {word_accuracy:.2f}% (Good)", "Feedback saved."
        else:
            return transcription, "No reference text provided.", ""
    except Exception as e:
        return f"Error during transcription: {e}", "", ""

# ---------------------------
# GRADIO UI
# ---------------------------
with gr.Blocks(title="Tanglish ASR Evaluation Dashboard") as iface:
    gr.Markdown(
        f"""
        # 🎙️ Tanglish ASR → English Translation
        Upload or record Tamil-English (Tanglish) speech.
        The **Whisper model ({MODEL_NAME})** transcribes and translates it.
        You can compare the transcription with your expected English text.
        Accuracy (WAcc) > 80% is considered good ✅
        """
    )

    with gr.Row():
        audio_input = gr.Audio(
            sources=["microphone", "upload"],
            type="filepath",
            label="🎧 Record or Upload Audio"
        )

    reference_text = gr.Textbox(
        label="Enter Expected English Text (for WER/WAcc calc)",
        placeholder="Type the reference transcription..."
    )

    with gr.Row():
        transcription_output = gr.Textbox(label="📝 English Transcription", interactive=False)
        accuracy_output = gr.Textbox(label="📊 WER/WAcc Result", interactive=False)


    btn_run = gr.Button("🚀 Transcribe & Evaluate")

    btn_run.click(
        fn=transcribe_and_evaluate,
        inputs=[audio_input, reference_text],
        outputs=[transcription_output, accuracy_output]
    )

iface.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.0 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


✅ ASR model loaded: openai/whisper-medium (device: cuda:0)
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d7b398d341527b64e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


break 2 end